## Advanced Cleaning on Steam Games Data

In this notebook we try out two different recommenders on [Steam dataset on Kaggle](https://www.kaggle.com/tamber/steam-video-games). We will use:
- ```numpy, pandas, sklearn``` for data preprocessing

The dataset has no header but comes in 5 columns:
- User ID: integer
- Game title: string
- Activity: string (purchase/play)
- Status: float (1.0 if activity is purchase, total number of hours in game if activity is play)
- A column full of 0's, will discard

In [1]:
import pandas as pd
import numpy as np 
import matplotlib
# import matplotlib.pyplot as plt 
import seaborn as sns
import turicreate
import sklearn as sk
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

In [2]:
steam_data = pd.read_pickle("clean_steam_data.pkl")
steam_data.head()
data_copy = steam_data

### we can easily get basic information about the data in each column 

In [3]:
data_copy.describe()

,userId,purchase,play,Actions
count,1.295350e+05,129535.0,70785.0,70785.000000
mean,1.024410e+08,1.0,1.0,48.770761
std,7.236206e+07,0.0,0.0,228.927258
min,5.250000e+03,1.0,1.0,0.100000
25%,4.548346e+07,1.0,1.0,1.000000
50%,8.605570e+07,1.0,1.0,4.500000
75%,1.542307e+08,1.0,1.0,19.100000
max,3.099031e+08,1.0,1.0,11754.000000


### Some obervations

Above summary shows that ```purchase``` has only a single value 1.0. We will **not** include purchase status in building our recommender, for two reasons:
1. number of hours played is more interesting to look at - it is an implicit feedback of the user's preference for games he/she has purchased
2. if using [Jaccard similarity](https://apple.github.io/turicreate/docs/api/generated/turicreate.recommender.item_similarity_recommender.ItemSimilarityRecommender.html) when measuring the similarity between two sets of elements, the number of hours played are treated as binary purchase status (1/0)

For ```play``` column, the data is heavily skewed and has a large range. There are different ways to do normalization. Since we are going to use it to represent the user's ranking of preference within his / her own purchases, let's convert it to the percentage of hours each user spends on each game he / she owns. 

But before doing that, notice there are 128804 purchase records and only 70489 play records, meaning some users bought certain games but never played them. We need to find a score for those less-favoured games too. To differentiate them from the games that a user has never bought, we can impute the corresponding play fields with a small value less than the min 0.1, say 0.05. Then we can move on to calculate the percentage as planned.

In [4]:
data_copy.drop("purchase", axis = 1, inplace = True)
data_copy["Actions"].fillna(0.05, inplace=True)
data_copy["play"].fillna(0, inplace=True)


just in case there is duplicated row in the dataframe, we implement the following function to ensure that every row in our dataframe will be distinct

In [5]:
#remove duplicates 
data_copy = data_copy.drop_duplicates(subset=None,keep="first")

besides, we could also have duplicated rows with the same userId and gameName appearing together. This might be some mistakes that happen during the data collection stage so that the hours has not been fully merged and accumulated. we wanna sum up the number of hours for the corrections

In [6]:
dp = data_copy[data_copy.duplicated(subset=['userId','gameName'], keep=False)]
dp = dp.groupby(['userId','gameName'], as_index=False)['Actions'].sum()
print(dp)
dp = dp.groupby(['userId','gameName'], as_index=False)['Actions'].sum()
data_copy.update(dp)
data_copy = data_copy.drop_duplicates(subset=['userId','gameName'],keep="first")

       userId                                      gameName  Actions
0    28472068                          Grand Theft Auto III      0.5
1    28472068                  Grand Theft Auto San Andreas      0.9
2    28472068                    Grand Theft Auto Vice City      5.7
3    33865373                   Sid Meier's Civilization IV    137.0
4    50769696                  Grand Theft Auto San Andreas     14.0
5    59925638                       Tom Clancy's H.A.W.X. 2      7.4
6    71411882                          Grand Theft Auto III      1.3
7    71510748                  Grand Theft Auto San Andreas      0.8
8   118664413                  Grand Theft Auto San Andreas      2.1
9   148362155                  Grand Theft Auto San Andreas     26.3
10  176261926                   Sid Meier's Civilization IV     14.8
11  176261926  Sid Meier's Civilization IV Beyond the Sword    564.4


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[col] = expressions.where(mask, this, that)


### Replace actual hours played with percentage of hours on a game for each user

In [16]:
d = data_copy.groupby('userId')['Actions'].apply(lambda x: x/ x.sum())
data_copy.update(d)
# convert userId to int
data_copy['userId'] = data_copy['userId'].astype(int)

### Output and save the dataframe

In [8]:
data_copy.to_pickle('clean_steam.pkl')

### Before building the recommender, here is an optional step - indexing the game names with integers
Based on doc of turicreate.recommender.create, the user ID and item ID columns can be either int or str, meaning that keeping the game title as is should be fine. Here let's do an extra step that indexes those strings with integers. (Maybe when we feel less lazy we can try spark.ml's recommender on it too.)

However indexing is essential when you build models with Spark's machine learning libraries.

In [9]:
data_copy['gameId']= data_copy.gameName.astype('category').cat.codes

### we want to create a look up table for the gameName and gameId columns

In [10]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128792 entries, 41879 to 83172
Data columns (total 5 columns):
userId      128792 non-null float64
gameName    128792 non-null object
play        128792 non-null float64
Actions     128792 non-null float64
gameId      128792 non-null int16
dtypes: float64(3), int16(1), object(1)
memory usage: 5.2+ MB


In [11]:
game_map = data_copy[['gameName','gameId']].copy().drop_duplicates().sort_values(by=['gameId'])
game_map.head()

,gameName,gameId
40639,007 Legends,0
72197,0RBITALIS,1
58190,1... 2... 3... KICK IT! (Drop That Beat Like a...,2
72198,10 Second Ninja,3
47824,"10,000,000",4


### Item - Item similarity model in Turicreate

According to the documentation [here](https://apple.github.io/turicreate/docs/userguide/recommender/using-trained-models.html), turicreate excludes the items that are observed for each user. In order to test the accuracy of the model, it is important to split a proportion of user's data into train and test for validation. 
but for now, let us just feed all the data in to our model.

There are 3 similarity measurements for similarity type - jaccard, cosine and pearson. In our case, We choose Pearson to measure the loss.

In [17]:
# create SFrame from DataFrame
from turicreate import SFrame

game_sf = SFrame(data=data_copy)

item_item_reco = turicreate.recommender.item_similarity_recommender.create( \
    game_sf, user_id='userId', item_id='gameId', \
    target="Actions", similarity_type='pearson')

Warning: Ignoring columns gameName, play;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 128792 observations with 12393 users and 5155 items.

Data prepared in: 0.146639s

...

| 873.734ms                           | 100              | 5155            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.94106s

In [19]:
# Get the k most similar items for each item in items. Default number is 10.
similar_games = item_item_reco.get_similar_items()

In [41]:
# for example to check some results
game_map_sframe = turicreate.SFrame(game_map)

similar_games_named = similar_games.join(game_map_sframe, on="gameId", how="left") \
   .join(game_map_sframe, on={"similar":"gameId"}, how="left") \
  .rename({"gameName.1":"similar_game"}, True) \
   .select_columns(["gameId", "gameName", "similar", "similar_game", "score", "rank"])

# print the result of one game for testing
a =similar_games_named[similar_games_named['gameName']== "Half-Life 2 Episode One"]
a.print_rows(num_rows=20, num_columns=6) 


+--------+-------------------------+---------+-------------------------------+
| gameId |         gameName        | similar |          similar_game         |
+--------+-------------------------+---------+-------------------------------+
|  2072  | Half-Life 2 Episode One |   2070  |          Half-Life 2          |
|  2072  | Half-Life 2 Episode One |   2077  |  Half-Life Deathmatch Source  |
|  2072  | Half-Life 2 Episode One |   2073  |    Half-Life 2 Episode Two    |
|  2072  | Half-Life 2 Episode One |   2074  |     Half-Life 2 Lost Coast    |
|  2072  | Half-Life 2 Episode One |   3627  |       SEGA Bass Fishing       |
|  2072  | Half-Life 2 Episode One |   1003  |           Crazy Taxi          |
|  2072  | Half-Life 2 Episode One |   480   | Battlefield Bad Company 2 ... |
|  2072  | Half-Life 2 Episode One |   3222  |             Portal            |
|  2072  | Half-Life 2 Episode One |   1336  |             Dota 2            |
|  2072  | Half-Life 2 Episode One |   2076  |      

### Let's pick a user and see what recommendations he / she gets:

In [43]:
item_item_reco.recommend().join(game_map_sframe, on="gameId", how="left")

recommendations finished on 1000/12393 queries. users per second: 24980

recommendations finished on 2000/12393 queries. users per second: 27511.1

recommendations finished on 3000/12393 queries. users per second: 29197.6

recommendations finished on 4000/12393 queries. users per second: 30571

recommendations finished on 5000/12393 queries. users per second: 27673.4

...

recommendations finished on 9000/12393 queries. users per second: 25202.3

recommendations finished on 10000/12393 queries. users per second: 25888.7

recommendations finished on 11000/12393 queries. users per second: 25390.6

recommendations finished on 12000/12393 queries. users per second: 24999.6

userId,gameId,score,rank,gameName
5250,3364,1.0,1,RECYCLE
5250,1705,1.0,2,Fast & Furious Showdown
5250,2306,1.0,3,Jack Keane
5250,3533,1.0,4,Ride! Carnival Tycoon
5250,2265,1.0,5,Insane 2
5250,4042,1.0,6,Stargate Resistance
5250,1271,1.0,7,Diaper Dash
5250,1607,1.0,8,FIFA Manager 10
5250,4459,1.0,9,The Promised Land
5250,5061,1.0,10,Xpand Rally
